In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Read conversation data from CSV file
conversations_df = pd.read_csv('/content/drive/MyDrive/BIG_DATA/Language Model/personality.csv')
conversations_df = conversations_df.sample(n=800, random_state=42)
speakers = conversations_df['Persona'].tolist()
texts = conversations_df['chat'].tolist()

# Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1

# Create input sequences using sliding window approach
input_sequences = []
for text in texts:
    sequence = tokenizer.texts_to_sequences([text])[0]
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_length = max([len(sequence) for sequence in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

# Split input sequences into input and target
xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]
ys = to_categorical(labels, num_classes=total_words)

# Define the model
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(xs, ys, epochs=100, verbose=1)

# Generate text
seed_text = "Hello"
next_words = 50

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print("Generated Text:", seed_text)


Epoch 1/100
3700/3700 [==============================] - 129s 33ms/step - loss: 5.8534 - accuracy: 0.1112
Epoch 2/100
3700/3700 [==============================] - 59s 16ms/step - loss: 5.1489 - accuracy: 0.1797
Epoch 3/100
3700/3700 [==============================] - 55s 15ms/step - loss: 4.8630 - accuracy: 0.2032
Epoch 4/100
3700/3700 [==============================] - 53s 14ms/step - loss: 4.6593 - accuracy: 0.2133
Epoch 5/100
3700/3700 [==============================] - 54s 15ms/step - loss: 4.4973 - accuracy: 0.2208
Epoch 6/100
3700/3700 [==============================] - 54s 15ms/step - loss: 4.3408 - accuracy: 0.2292
Epoch 7/100
3700/3700 [==============================] - 53s 14ms/step - loss: 4.2075 - accuracy: 0.2343
Epoch 8/100
3700/3700 [==============================] - 55s 15ms/step - loss: 4.0863 - accuracy: 0.2403
Epoch 9/100
3700/3700 [==============================] - 54s 15ms/step - loss: 3.9762 - accuracy: 0.2470
Epoch 10/100
3700/3700 [==============================

In [6]:
# Generate text
seed_text = "Hello, do you know me"
next_words = 50

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print("Generated Text:", seed_text)

1/1 [==============================] - 0s 20ms/step
Generated Text: Hello, do you know me just bring some place my cats and two kids hello i am a student i am in school i am a nerd i am sorry to hear that i have been to the way every day i am sorry i am not studying i am sorry i am a lawyer


In [7]:
model.save('trained_model.h5')